In [3]:
import numpy as np
import cv2
import scipy.io
import matplotlib.pyplot as plt
import glob
from scipy import ndimage as ndi

import sys
sys.path.append('..')
sys.path.append('../pipeline')
from pipeline.dl.viz_utils import *

In [5]:
def write_imgs(npy_imgs, outdir, fold="train"):
    for i in range(npy_imgs.shape[0]):
        fn = f"{outdir}/{fold}_{i}.png"
        cv2.imwrite(fn, npy_imgs[i, ...])

# Load images in .npy format
fold1_imgs_dir = "../../../datasets/Nucleisegmentation-PanNuke/orig/Fold 1/images/fold1/images.npy"
fold2_imgs_dir = "../../../datasets/Nucleisegmentation-PanNuke/orig/Fold 2/images/fold2/images.npy"
fold3_imgs_dir = "../../../datasets/Nucleisegmentation-PanNuke/orig/Fold 3/images/fold3/images.npy"
fold1_imgs = np.load(fold1_imgs_dir).astype('uint8')
fold2_imgs = np.load(fold2_imgs_dir).astype('uint8')
fold3_imgs = np.load(fold3_imgs_dir).astype('uint8')

# Write images to .png to corresponding dirs
train_imgs_dir = "../../../datasets/Nucleisegmentation-PanNuke/train/Images"
valid_imgs_dir = "../../../datasets/Nucleisegmentation-PanNuke/valid/Images"
test_imgs_dir = "../../../datasets/Nucleisegmentation-PanNuke/test/Images"

write_imgs(fold1_imgs, train_imgs_dir, fold="train")
write_imgs(fold2_imgs, valid_imgs_dir, fold="valid")
write_imgs(fold3_imgs, test_imgs_dir, fold="test")

In [24]:
def write_masks(npy_masks, outdir, fold="train"):
    for i in range(npy_masks.shape[0]):
        t = npy_masks[i, ...]
        inst_map = np.zeros(t.shape[:2], dtype=np.uint32)
        for l in range(t.shape[-1]-1):
            inst_map += t[..., l]
        fn = f"{outdir}/{fold}_{i}.mat"
        scipy.io.savemat(fn, mdict={'inst_map': inst_map})

# load masks in the .npy format
fold1_masks_dir = "../../../datasets/Nucleisegmentation-PanNuke/orig/Fold 1/masks/fold1/masks.npy"
fold2_masks_dir = "../../../datasets/Nucleisegmentation-PanNuke/orig/Fold 2/masks/fold2/masks.npy"
fold3_masks_dir = "../../../datasets/Nucleisegmentation-PanNuke/orig/Fold 3/masks/fold3/masks.npy"        
fold1_masks = np.load(fold1_masks_dir).astype('uint32')
fold2_masks = np.load(fold2_masks_dir).astype('uint32')
fold3_masks = np.load(fold3_masks_dir).astype('uint32')

# specify output dirs
train_masks_dir = "../../../datasets/Nucleisegmentation-PanNuke/train/Labels"
valid_masks_dir = "../../../datasets/Nucleisegmentation-PanNuke/valid/Labels"
test_masks_dir = "../../../datasets/Nucleisegmentation-PanNuke/test/Labels"
    
# write masks to corresponding directories as .mat format
write_masks(fold1_masks, train_masks_dir, fold="train")
write_masks(fold2_masks, valid_masks_dir, fold="valid")
write_masks(fold3_masks, test_masks_dir, fold="test")

In [36]:
def write_overlays(gts, ims, outdir, fold="train"):
    for j, (g, i) in enumerate(zip(gts, ims)):
        gt = scipy.io.loadmat(g)
        gt = gt['inst_map']
        im = cv2.imread(i)
        _, overlay = draw_contours(gt, im) 
        fn = f"{dir_train_ol}/{fold}_{j}_overlay.png"
        cv2.imwrite(fn, overlay)
        

# specify output dirs
dir_train_ol = "../../../datasets/Nucleisegmentation-PanNuke/train/Overlay_binary"
dir_valid_ol = "../../../datasets/Nucleisegmentation-PanNuke/valid/Overlay_binary"
dir_test_ol = "../../../datasets/Nucleisegmentation-PanNuke/test/Overlay_binary"

# List the image and gt files
train_gts = sorted(glob.glob(train_mask_dir + "/*.mat"))
train_ims = sorted(glob.glob(dir_train + "/*.png"))
valid_gts = sorted(glob.glob(valid_mask_dir + "/*.mat"))
valid_ims = sorted(glob.glob(dir_valid + "/*.png"))
test_gts = sorted(glob.glob(test_mask_dir + "/*.mat"))
test_ims = sorted(glob.glob(dir_test + "/*.png"))

# write mask image overlays to corresponding directories as .png
write_overlays(train_gts, train_ims, dir_train_ol, fold="train")
write_overlays(valid_gts, valid_ims, dir_valid_ol, fold="valid")
write_overlays(train_gts, train_ims, dir_test_ol, fold="test")